In [1]:

import pandas as pd
import scipy
import pyodbc

import time

conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=LAPTOP-IFK6D8L3\\SQLEXPRESS;DATABASE=StockQuote;UID=sa;PWD=password')
cur = conn.cursor()


SQL_NLM_250 = """

select Script_Name from Index_Stock where Index_Name ='Niftylargemidcap250 ' 
"""
df_NLM250 = pd.read_sql(SQL_NLM_250,conn)

display(df_NLM250)

ScriptList = df_NLM250['Script_Name'].to_list()
# dflist = {}
for s in ScriptList:    
    df = pd.DataFrame()
    SQL = """ select Script_Name, Trnx_date trnx, [open], [high], [low], [volume], [close]cls from NSE_EOD where Trnx_date >= '2021-01-01' and Script_Name = '{}' order by Script_Name,Trnx_date """.format(s.strip())
    print(SQL)
    df = pd.read_sql(SQL,conn)
    # dflist[s.strip()] = df
    df['Gain'] = df['cls']-df['open']
    
    def ewm(df, days):
        
        df_gby = (
            (df
                .groupby(['Script_Name', 'trnx'])['cls']
                .sum()
            )
            .ewm(span=days, min_periods=days)
            .mean()
            .droplevel(1)
            .reset_index(name='ema'+str(days))
        )
    
        return df_gby['ema'+str(days)]

    for i in [5,9,12,21,25,26,50,100,200]:
        df['ema'+str(i)] = ewm(df, i)


    df['MACD'] = df['ema12']-df['ema26']
    display(df)
    #df['Signal'] = ewm(df['MACD'],9)
    df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACDS'] = df['MACD']-df['Signal']
    # df['US'] = df['Gain'].apply(lambda x: df['high']-df['cls'] if x>=0 else df['high']-df['open'])
    # df['LS'] = df['Gain'].apply(lambda x: (df['open']-df['low']) if x>=0 else (df['cls']-df['low']))
    # df['LS'] = df['Gain'].apply(lambda x: df.assign(LS = lambda x: (x['open'] - x['low'])) if x>=0 else df.assign(LS = lambda x: (x['cls'] - x['low'])))
    df.to_csv("./{}.csv".format(s.strip()))

    


# df_gby

,Script_Name
0,3MINDIA
1,AAVAS
2,ABB
3,ABBOTINDIA
4,ABCAPITAL
...,...
245,YESBANK
246,ZEEL
247,ZFCVINDIA
248,ZOMATO


 select Script_Name, Trnx_date trnx, [open], [high], [low], [volume], [close]cls from NSE_EOD where Trnx_date >= '2021-01-01' and Script_Name = '3MINDIA' order by Script_Name,Trnx_date 


,Script_Name,trnx,open,high,low,volume,cls,Gain,ema5,ema9,ema12,ema21,ema25,ema26,ema50,ema100,ema200,MACD
0,3MINDIA,2021-01-04,21288.00,21300.00,21065.00,3043,21154.65,-133.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3MINDIA,2021-01-05,21154.65,21899.00,21011.00,3048,21594.20,439.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3MINDIA,2021-01-06,21775.00,22100.00,21200.00,8558,21458.40,-316.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3MINDIA,2021-01-07,21650.00,21700.00,21435.10,3515,21626.00,-24.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3MINDIA,2021-01-08,21553.00,21898.95,21331.00,15435,21735.40,182.40,21600.042654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,3MINDIA,2022-12-20,23240.00,23240.00,22860.05,695,22917.90,-322.10,23085.290094,23129.685852,23152.226540,23210.536207,23231.517910,23236.192359,23268.669153,23052.779900,22856.962780,-83.965819
485,3MINDIA,2022-12-21,23139.00,23229.00,22750.00,6291,22873.60,-265.40,23014.726729,23078.468682,23109.360918,23179.905643,23203.985763,23209.333666,23253.176245,23049.231570,22857.129617,-99.972747
486,3MINDIA,2022-12-22,22993.00,22993.00,22500.00,4890,22622.50,-370.50,22883.984486,22987.274946,23034.459239,23129.232403,23159.256089,23165.864505,23228.443844,23040.780942,22854.776942,-131.405267
487,3MINDIA,2022-12-23,22622.50,23000.00,22150.00,2355,22424.25,-198.25,22730.739657,22874.669956,22940.580894,23065.143093,23102.717159,23110.930098,23196.906830,23028.571704,22850.460300,-170.349203


 select Script_Name, Trnx_date trnx, [open], [high], [low], [volume], [close]cls from NSE_EOD where Trnx_date >= '2021-01-01' and Script_Name = 'AAVAS' order by Script_Name,Trnx_date 


,Script_Name,trnx,open,high,low,volume,cls,Gain,ema5,ema9,ema12,ema21,ema25,ema26,ema50,ema100,ema200,MACD
0,AAVAS,2021-01-04,1748.80,1939.00,1673.60,79683,1867.05,118.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAVAS,2021-01-05,1872.00,2117.95,1872.00,266074,2021.70,149.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAVAS,2021-01-06,2040.90,2053.00,1886.10,226683,1910.30,-130.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAVAS,2021-01-07,1929.20,1930.00,1871.00,119379,1891.30,-37.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAVAS,2021-01-08,1896.00,1948.00,1896.00,148479,1933.10,37.10,1923.581517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,AAVAS,2022-12-20,1973.05,2029.90,1961.30,153377,1978.75,5.70,1970.907342,1959.689936,1954.172359,1950.688532,1953.354709,1954.286933,1988.972282,2061.811265,2177.619209,-0.114574
485,AAVAS,2022-12-21,1988.30,2000.00,1962.00,99122,1975.45,-12.85,1972.421561,1962.841949,1957.445842,1952.939574,1955.054346,1955.854568,1988.441996,2060.101038,2175.591863,1.591274
486,AAVAS,2022-12-22,1961.95,1982.45,1901.20,170388,1920.90,-41.05,1955.247708,1954.453559,1951.823405,1950.026886,1952.427089,1953.265340,1985.793290,2057.344419,2173.038020,-1.441936
487,AAVAS,2022-12-23,1903.40,1907.40,1865.00,50501,1871.95,-31.45,1927.481805,1937.952847,1939.535189,1942.928987,1946.236544,1947.241982,1981.328848,2053.673031,2170.019186,-7.706793


...

,Script_Name,trnx,open,high,low,volume,cls,Gain,ema5,ema9,ema12,ema21,ema25,ema26,ema50,ema100,ema200,MACD
0,ZFCVINDIA,2022-04-01,7717.00,7747.95,7452.00,931,7616.20,-100.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZFCVINDIA,2022-04-04,7715.90,7777.00,7560.00,1074,7685.40,-30.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZFCVINDIA,2022-04-05,7659.05,8000.00,7642.10,4889,7934.00,274.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ZFCVINDIA,2022-04-06,8000.00,8050.05,7800.00,5822,7931.90,-68.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ZFCVINDIA,2022-04-07,7826.20,8093.95,7821.10,3868,7900.95,74.75,7868.399763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,ZFCVINDIA,2022-12-20,9227.10,9250.00,9170.00,2028,9198.00,-29.10,9221.774231,9261.433135,9291.370716,9380.664102,9417.396734,9425.991895,9537.186579,9438.435029,NaN,-134.621180
178,ZFCVINDIA,2022-12-21,9186.00,9250.00,9000.00,6586,9080.20,-105.80,9174.582821,9225.186508,9258.882913,9353.349183,9391.458509,9400.377654,9519.251614,9431.137877,NaN,-141.494741
179,ZFCVINDIA,2022-12-22,9140.00,9140.00,8810.00,7259,8860.50,-279.50,9069.888547,9152.249206,9197.593234,9308.544710,9350.615524,9360.386678,9493.398934,9419.520731,NaN,-162.793444
180,ZFCVINDIA,2022-12-23,8852.85,8898.30,8638.55,13100,8767.40,-85.45,8969.059031,9075.279365,9131.409660,9259.349735,9305.752768,9316.461700,9464.907971,9406.252123,NaN,-185.052040


 select Script_Name, Trnx_date trnx, [open], [high], [low], [volume], [close]cls from NSE_EOD where Trnx_date >= '2021-01-01' and Script_Name = 'ZOMATO' order by Script_Name,Trnx_date 


,Script_Name,trnx,open,high,low,volume,cls,Gain,ema5,ema9,ema12,ema21,ema25,ema26,ema50,ema100,ema200,MACD
0,ZOMATO,2021-07-23,116.00,138.90,115.00,694895290,126.00,10.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZOMATO,2021-07-26,126.35,143.75,125.30,249723854,140.65,14.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZOMATO,2021-07-27,141.70,147.80,127.75,240341900,132.90,-8.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ZOMATO,2021-07-28,131.00,135.00,123.55,159793731,131.20,0.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ZOMATO,2021-07-29,134.95,144.00,132.20,117973089,141.55,6.60,136.143839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,ZOMATO,2022-12-20,62.10,63.20,61.90,32487667,62.85,0.75,63.082159,63.566603,63.820423,64.235918,64.308176,64.318550,64.113016,64.542707,70.418353,-0.498126
349,ZOMATO,2022-12-21,63.20,63.50,60.85,39979212,61.35,-1.85,62.504772,63.123282,63.440358,63.973562,64.080624,64.098657,64.004662,64.479427,70.325311,-0.658299
350,ZOMATO,2022-12-22,61.80,62.05,58.55,53936114,58.95,-2.85,61.319848,62.288626,62.749534,63.516874,63.685961,63.717275,63.806440,64.369836,70.208636,-0.967741
351,ZOMATO,2022-12-23,58.30,58.35,53.20,93022114,53.65,-4.65,58.763232,60.560901,61.349606,62.619886,62.913964,62.971551,63.408148,64.157375,70.038848,-1.621945


 select Script_Name, Trnx_date trnx, [open], [high], [low], [volume], [close]cls from NSE_EOD where Trnx_date >= '2021-01-01' and Script_Name = 'ZYDUSLIFE' order by Script_Name,Trnx_date 


,Script_Name,trnx,open,high,low,volume,cls,Gain,ema5,ema9,ema12,ema21,ema25,ema26,ema50,ema100,ema200,MACD
0,ZYDUSLIFE,2022-03-07,343.70,345.75,331.55,969180,342.10,-1.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZYDUSLIFE,2022-03-08,342.10,347.00,335.75,978348,344.50,2.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZYDUSLIFE,2022-03-09,350.00,363.00,348.05,3946522,359.35,9.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ZYDUSLIFE,2022-03-10,368.00,368.00,361.50,2097821,365.75,-2.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ZYDUSLIFE,2022-03-11,363.10,374.60,362.40,1365084,370.65,7.55,362.328673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,ZYDUSLIFE,2022-12-20,406.55,412.50,404.75,789245,411.35,4.80,409.528325,409.076345,408.988353,409.510942,409.720879,409.749972,407.598314,398.519145,NaN,-0.761619
196,ZYDUSLIFE,2022-12-21,410.60,426.00,410.30,2697599,421.90,11.30,413.652216,411.641076,410.974761,410.637220,410.657735,410.649975,408.159377,398.991314,NaN,0.324786
197,ZYDUSLIFE,2022-12-22,427.95,427.95,416.25,1770189,423.55,-4.40,416.951478,414.022861,412.909413,411.811110,411.649448,411.605532,408.763150,399.487074,NaN,1.303881
198,ZYDUSLIFE,2022-12-23,423.50,430.00,418.90,2075717,420.30,-3.20,418.067652,415.278289,414.046426,412.582827,412.314875,412.249567,409.215733,399.907058,NaN,1.796859


In [35]:
display(dflist['ASIANPAINT'])

,Script_Name,trnx,cls
0,ASIANPAINT,2021-01-04,2753.70
1,ASIANPAINT,2021-01-05,2793.85
2,ASIANPAINT,2021-01-06,2805.35
3,ASIANPAINT,2021-01-07,2792.25
4,ASIANPAINT,2021-01-08,2844.70
...,...,...,...
484,ASIANPAINT,2022-12-20,3082.15
485,ASIANPAINT,2022-12-21,3069.65
486,ASIANPAINT,2022-12-22,3088.55
487,ASIANPAINT,2022-12-23,3057.90


In [24]:
ScriptList = df_NLM250['Script_Name'].to_list()
for s in ScriptList:
    print(s)
    


3MINDIA             
AAVAS               
ABB                 
ABBOTINDIA          
ABCAPITAL           
ABFRL               
ACC                 
ADANIENT            
ADANIGREEN          
ADANIPORTS          
ADANITRANS          
AFFLE               
AIAENG              
AJANTPHARM          
ALKEM               
ALKYLAMINE          
AMBUJACEM           
APLAPOLLO           
APLLTD              
APOLLOHOSP          
APOLLOTYRE          
ASHOKLEY            
ASIANPAINT          
ASTRAL              
ATGL                
ATUL                
AUBANK              
AUROPHARMA          
AXISBANK            
BAJAJ-AUTO          
BAJAJFINSV          
BAJAJHLDNG          
BAJFINANCE          
BALKRISIND          
BANDHANBNK          
BANKBARODA          
BANKINDIA           
BATAINDIA           
BAYERCROP           
BEL                 
BERGEPAINT          
BHARATFORG          
BHARTIARTL          
BHEL                
BIOCON              
BLUEDART            
BOSCHLTD            
BPCL         

In [6]:
df_gby = (
        (df
            .groupby(['Script_Name', 'trnx'])['cls']
            .mean()
        )
        .ewm(span=12)
        .mean()
        # .droplevel(2)
        # .reset_index(name='ema'+str(days))
    )

df_gby

Script_Name  trnx      
ZYDUSWELL    2021-01-04    2148.400000
             2021-01-05    2129.550000
             2021-01-06    2112.337760
             2021-01-07    2096.306049
             2021-01-08    2087.963325
                              ...     
             2022-12-20    1598.901145
             2022-12-21    1594.754815
             2022-12-22    1586.715613
             2022-12-23    1572.382442
             2022-12-26    1565.531297
Name: cls, Length: 489, dtype: float64

In [7]:
def ewm(df, days):
    df_gby = (
        (df
            .groupby(['Script_Name', 'trnx'])['cls']
            .sum()
        )
        .ewm(span=days, min_periods=days)
        .mean()
        .droplevel(1)
        .reset_index(name='ema'+str(days))
    )
    
    return df_gby['ema'+str(days)]

In [8]:
for i in [5,9,12,21,25,26,50,100,200]:
    df['ema'+str(i)] = ewm(df, i)

df

,Script_Name,trnx,cls,ema5,ema9,ema12,ema21,ema25,ema26,ema50,ema100,ema200,MACD
0,ZYDUSWELL,2021-01-04,2148.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZYDUSWELL,2021-01-05,2113.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZYDUSWELL,2021-01-06,2085.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ZYDUSWELL,2021-01-07,2061.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ZYDUSWELL,2021-01-08,2065.60,2079.688626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,ZYDUSWELL,2022-12-20,1588.85,1594.858946,1597.727345,1598.901145,1604.996842,1608.531555,1609.414895,1624.372691,1632.680149,1665.327103,-10.513750
485,ZYDUSWELL,2022-12-21,1571.95,1587.222630,1592.571876,1594.754815,1601.992583,1605.717589,1606.639718,1622.316899,1631.477499,1664.390720,-11.884903
486,ZYDUSWELL,2022-12-22,1542.50,1572.315087,1582.557501,1586.715613,1596.584167,1600.854697,1601.888628,1619.186825,1629.715465,1663.168499,-15.173015
487,ZYDUSWELL,2022-12-23,1493.55,1546.060058,1564.756001,1572.382442,1587.217424,1592.600490,1593.863544,1614.259891,1627.018963,1661.467834,-21.481103


In [9]:
df['MACD'] = df['ema12']-df['ema26']
display(df)
#df['Signal'] = ewm(df['MACD'],9)
df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

,Script_Name,trnx,cls,ema5,ema9,ema12,ema21,ema25,ema26,ema50,ema100,ema200,MACD
0,ZYDUSWELL,2021-01-04,2148.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZYDUSWELL,2021-01-05,2113.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZYDUSWELL,2021-01-06,2085.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ZYDUSWELL,2021-01-07,2061.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ZYDUSWELL,2021-01-08,2065.60,2079.688626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,ZYDUSWELL,2022-12-20,1588.85,1594.858946,1597.727345,1598.901145,1604.996842,1608.531555,1609.414895,1624.372691,1632.680149,1665.327103,-10.513750
485,ZYDUSWELL,2022-12-21,1571.95,1587.222630,1592.571876,1594.754815,1601.992583,1605.717589,1606.639718,1622.316899,1631.477499,1664.390720,-11.884903
486,ZYDUSWELL,2022-12-22,1542.50,1572.315087,1582.557501,1586.715613,1596.584167,1600.854697,1601.888628,1619.186825,1629.715465,1663.168499,-15.173015
487,ZYDUSWELL,2022-12-23,1493.55,1546.060058,1564.756001,1572.382442,1587.217424,1592.600490,1593.863544,1614.259891,1627.018963,1661.467834,-21.481103


In [10]:
display(df)

,Script_Name,trnx,cls,ema5,ema9,ema12,ema21,ema25,ema26,ema50,ema100,ema200,MACD,Signal
0,ZYDUSWELL,2021-01-04,2148.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZYDUSWELL,2021-01-05,2113.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZYDUSWELL,2021-01-06,2085.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ZYDUSWELL,2021-01-07,2061.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ZYDUSWELL,2021-01-08,2065.60,2079.688626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,ZYDUSWELL,2022-12-20,1588.85,1594.858946,1597.727345,1598.901145,1604.996842,1608.531555,1609.414895,1624.372691,1632.680149,1665.327103,-10.513750,-12.883639
485,ZYDUSWELL,2022-12-21,1571.95,1587.222630,1592.571876,1594.754815,1601.992583,1605.717589,1606.639718,1622.316899,1631.477499,1664.390720,-11.884903,-12.683892
486,ZYDUSWELL,2022-12-22,1542.50,1572.315087,1582.557501,1586.715613,1596.584167,1600.854697,1601.888628,1619.186825,1629.715465,1663.168499,-15.173015,-13.181716
487,ZYDUSWELL,2022-12-23,1493.55,1546.060058,1564.756001,1572.382442,1587.217424,1592.600490,1593.863544,1614.259891,1627.018963,1661.467834,-21.481103,-14.841594


In [23]:
df.to_csv("./test11.csv")